In [18]:
import time
import requests
import pandas as pd
import numpy as np
import itertools
import datetime
import re
import nltk

from collections import Counter
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
pat_num = pd.read_csv('./data/google_health_care_patent.csv', encoding='utf8')
pat_nums = [i.split('-')[1] for i in pat_num['id']]

In [3]:
url_post = 'https://api.patentsview.org/patents/query'
post_len = 10000

# Crawling elements (인용, 피인용 grant date 포함)
f = '&f=["patent_number","patent_title","patent_date", '\
'"patent_type","patent_abstract","cpc_subgroup_id", '\
'"assignee_id", "assignee_total_num_patents", "cpc_total_num_assignees", '\
'"cited_patent_number", "cited_patent_date", "citedby_patent_number", "citedby_patent_date", '\
'"examiner_id", "examiner_role"]'

# # Crawling elements (인용, 피인용 grant date 미포함)
# f = '&f=["patent_number","patent_title","patent_date", '\
# '"patent_type","patent_abstract","cpc_subgroup_id", '\
# '"assignee_id", "assignee_total_num_patents", "cpc_total_num_assignees", '\
# '"cited_patent_number", "citedby_patent_number", '\
# '"examiner_id", "examiner_role"]'

In [ ]:
pt_dict_list=[]
st = time.time()
for n, i in enumerate(tqdm(pat_nums)):
    q1 = '?q={"patent_number":'
    q2 = '}'
    q = q1 + '"' + i + '"' + q2
    
    t_data = q + f
    t_post = requests.get(url_post + t_data)
    
    while '500' in str(t_post):
        t_post = requests.get(url_post + t_data)
    try:
        t_json = t_post.json()
    except:
        continue
        
    try:
        pt_dict_list.extend(t_json['patents'])
    except TypeError:
        pass
    
#     if n%10 == 0:
#         print(n / 10)
        
print('Time spending', time.time() - st)

In [ ]:
# CPC to list
for d in util_patent:
    cpc_set = ''
    for c in d['cpcs']:
        try:
            cpc = c['cpc_subgroup_id'].split('/')[0]
            cpc_set = cpc_set + ',' + cpc
        except:
            cpc_set = cpc_set + 'None'
    d['cpc_set'] = cpc_set

In [ ]:
### cpc set 리스트로
for c in util_patent:
    cc = list(c['cpc_set'].split(','))
    cc = list(filter(None, cc))
    c['cpc_set'] = cc

In [30]:
# Extract backward citations
for d in util_patent:
    cited_patent_set = ''
    for c in d['cited_patents']:
        try:
            cited = c['cited_patent_number']
            cited_patent_set = cited_patent_set + ',' + cited
        except:
            cited_patent_set = cited_patent_set + 'None'
    d['cited_patent_set'] = cited_patent_set

In [ ]:
# prior knowledge

# Count number of backward citations
for d in util_patent:
    num_cited=[]
    for c in d['cited_patents']:
        try:
            cited = c['cited_patent_number']
            num_cited.append(cited)
        except:
            num_cited.append('None')
    
    if 'None' in num_cited:
        d['num_cited'] = 0
    else:
        d['num_cited'] = len(num_cited)

In [ ]:
# Extract forward citations
for d in util_patent:
    citedby_patent_set = ''
    for c in d['citedby_patents']:
        try:
            citedby = c['citedby_patent_number']
            citedby_patent_set = citedby_patent_set + ',' + citedby
        except:
            citedby_patent_set = citedby_patent_set + 'None'
    d['citedby_patent_set'] = citedby_patent_set

In [ ]:
# Count number of forward citations
for d in util_patent:
    num_citedby=[]
    for c in d['citedby_patents']:
        try:
            citedby = c['citedby_patent_number']
            num_citedby.append(citedby)
        except:
            num_citedby.append('None')
    
    if 'None' in num_citedby:
        d['num_citedby'] = 0
    else:
        d['num_citedby'] = len(num_citedby)

In [ ]:
# forward citations to list

for d in util_patent:
    
    if 'None' not in d['citedby_patent_set']:
        a = d['citedby_patent_set'].split(',')
                
    else:
        a = list()
        
    d['forward_cpcs'] = a

In [ ]:
# dict to dataframe
final_patent_df = pd.DataFrame.from_dict(util_patent).drop(['cpcs'], axis=1)
final_patent_df.drop_duplicates(['patent_abstract'], inplace=True)
print('Patent list length: ', len(df))

In [ ]:
# filtering by patent type (we need utility type patent only!!)
util_patent = [i for i in pt_dict_list if i['patent_type'] == 'utility']

In [ ]:
# dict to dataframe
final_patent_df = pd.DataFrame.from_dict(util_patent).drop(['cpcs'], axis=1)
final_patent_df.drop_duplicates(['patent_abstract'], inplace=True)
print('Patent list length: ', len(df))

In [ ]:
# CSV로 저장
final_patent_df.to_csv('./data/final_patent.csv', encoding='utf8', index=False)

In [ ]:
# Excel로 저장
final_patent_df.to_excel('./data/final_patent.xlsx', encoding='utf8', index=False)

In [124]:
from ast import literal_eval
for p in util_patent:
    try:
        p['citedby_patents'] = literal_eval(p['citedby_patents'])
    except SyntaxError:
        pass

In [125]:
# 모든 특허의 forward citation의 cpc를 합침

forward_pat_nums = []
for d in tqdm(util_patent):
    forward_pat_num = []
    for c in d['citedby_patents']:
        try:
            forward = c['citedby_patent_number']
            forward_pat_num.append(forward)
        except:
            forward = 'None'
            forward_pat_num.append(forward)
    forward_pat_nums.append(forward_pat_num)    

100%|█████████████████████████████████████████████████████████████████████████| 33217/33217 [00:00<00:00, 39183.30it/s]


In [ ]:
# 이중리스트 flatten 및 중복제거 (forward citation id별로 cpc만 조회할것이기때문)
import itertools
forward_pat_nums_list = list(itertools.chain(*forward_pat_nums))
all_forward_pat_nums = list(dict.fromkeys(forward_pat_nums_list)) # 순서 유지하면서 중복 제거
all_forward_pat_nums = [v for v in all_forward_pat_nums if v] # None 제거

In [19]:
# forward citation의 정보 수집
url_post = 'https://api.patentsview.org/patents/query'
post_len = 10000

# target patent의 forward citation의 정보 수집
f = '&f=["patent_number","patent_title", "patent_date",'\
'"patent_type","patent_abstract","cpc_subgroup_id"]'

In [20]:
forward_pt_list=[]
st = time.time()
for n, i in enumerate(tqdm(all_forward_pat_nums)):
    q1 = '?q={"patent_number":'
    q2 = '}'
    q = q1 + '"' + i + '"' + q2
    
    t_data = q + f
    t_post = requests.get(url_post + t_data)
    
    while '500' in str(t_post):
        t_post = requests.get(url_post + t_data)
    try:
        t_json = t_post.json()
    except:
        continue
        
    try:
        forward_pt_list.extend(t_json['patents'])
    except TypeError:
        pass
    
#     if n%10 == 0:
#         print(n / 10)
        
print('Time spending', time.time() - st)

100%|█████████████████████████████████████████████████████████████████████████| 95927/95927 [22:58:49<00:00,  1.16it/s]

Time spending 82729.34797477722


In [37]:
# filtering by patent type (we need utility type patent only!!)
forward_util_patent = [i for i in forward_pt_list if i['patent_type'] == 'utility']

In [38]:
# forward citation의 CPC to list
for d in forward_util_patent:
    cpc_set = ''
    for c in d['cpcs']:
        try:
            cpc = c['cpc_subgroup_id'].split('/')[0]
            cpc_set = cpc_set + ',' + cpc
        except:
            cpc_set = cpc_set + 'None'
    d['forward_cpc_set'] = cpc_set

In [ ]:
### forward cpc set 리스트로
for f in forward_util_patent:
    aaa = list(f['forward_cpc_set'].split(','))
    aaa = list(filter(None, aaa))
    f['forward_cpc_set'] = aaa

In [26]:
# dict to dataframe
forward_patent_df = pd.DataFrame.from_dict(forward_util_patent).drop(['cpcs'], axis=1)
forward_patent_df.drop_duplicates(['patent_abstract'], inplace=True)
print('Patent list length: ', len(df))

Patent list length:  68590


In [27]:
# CSV로 저장
forward_patent_df.to_csv('./data/forward_citation_cpcs.csv', encoding='utf8', index=False)

In [24]:
#############################################################

In [25]:
#############################################################

In [ ]:
# backward citations to list

for d in util_patent:
    
    if 'None' not in d['cited_patent_set']:
        a = d['cited_patent_set'].split(',')
                
    else:
        a = list()
        
    d['backward_cpcs'] = a

In [ ]:
# 모든 특허의 backward citation의 cpc를 합침

backward_pat_nums = []
for d in util_patent:
    backward_pat_num = []
    for c in d['cited_patents']:
        try:
            backward = c['cited_patent_number']
            backward_pat_num.append(backward)
        except:
            backward = 'None'
            backward_pat_num.append(backward)
    backward_pat_nums.append(backward_pat_num)    

In [ ]:
# 이중리스트 flatten 및 중복제거 (backward citation id별로 cpc만 조회할것이기때문)

backward_pat_nums_list = list(itertools.chain(*backward_pat_nums))
all_backward_pat_nums = list(dict.fromkeys(backward_pat_nums_list)) # 순서 유지하면서 중복 제거
all_backward_pat_nums = [v for v in all_backward_pat_nums if v] # None 제거

In [31]:
# backward citation의 정보 수집
url_post = 'https://api.patentsview.org/patents/query'
post_len = 10000

# target patent의 backward citation의 정보 수집
f = '&f=["patent_number","patent_type","cpc_subgroup_id"]'

In [32]:
backward_pt_list=[]
st = time.time()
for n, i in enumerate(tqdm(all_backward_pat_nums)):
    q1 = '?q={"patent_number":'
    q2 = '}'
    q = q1 + '"' + i + '"' + q2
    
    t_data = q + f
    t_post = requests.get(url_post + t_data)
    
    while '500' in str(t_post):
        t_post = requests.get(url_post + t_data)
    try:
        t_json = t_post.json()
    except:
        continue
        
    try:
        backward_pt_list.extend(t_json['patents'])
    except TypeError:
        pass
    
#     if n%10 == 0:
#         print(n / 10)
        
print('Time spending', time.time() - st)

100%|███████████████████████████████████████████████████████████████████████| 388038/388038 [93:10:46<00:00,  1.16it/s]

Time spending 335446.5075290203


In [33]:
# filtering by patent type (we need utility type patent only!!)
backward_util_patent = [i for i in backward_pt_list if i['patent_type'] == 'utility']

In [34]:
# backward citation의 CPC to list
for d in backward_util_patent:
    cpc_set = ''
    for c in d['cpcs']:
        try:
            cpc = c['cpc_subgroup_id'].split('/')[0]
            cpc_set = cpc_set + ',' + cpc
        except:
            cpc_set = cpc_set + 'None'
    d['backward_cpc_set'] = cpc_set

In [ ]:
### backward cpc set 리스트로
for b in backward_util_patent:
    bbb = list(b['backward_cpc_set'].split(','))
    bbb = list(filter(None, bbb))
    b['backward_cpc_set'] = bbb

In [ ]:
# CSV로 저장
backward_patent_df.to_csv('./data/backward_citation_cpcs.csv', encoding='utf8', index=False)

In [11]:
"""
(작업중)변수 설계 파일에서 작업하고 밑에 추가할 예정
"""

'\n(작업중)변수 설계 파일에서 작업하고 밑에 추가할 예정\n'

In [96]:
final_patent_df = pd.read_excel('./data/final_patent.xlsx')

In [97]:
util_patent = final_patent_df.to_dict('records')

In [21]:
backward_patent_df = pd.read_csv('./data/backward_citation_cpcs.csv', encoding='utf8')

In [22]:
backward_util_patent = backward_patent_df.to_dict('records')

In [23]:
forward_patent_df = pd.read_csv('./data/forward_citation_cpcs.csv', encoding='utf8')

C:\Users\Woojin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6,7,8,9,10,11,12,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [24]:
forward_util_patent = forward_patent_df.to_dict('records')

In [131]:
len(forward_pat_nums[0])

3

In [132]:
util_patent[0]

{'patent_number': 9811636,
 'patent_title': 'Connected health care system',
 'patent_date': '2017-11-07',
 'patent_type': 'utility',
 'patent_abstract': 'A connected health care system is provided with various devices for tracking and monitoring health statistics and behaviors including oral health, fitness, heart health, bone health, salivary diagnostics, diabetes, and further options that will be explored in the subsequent sections. The connected health care system is comprised of various devices, a data transfer medium (i.e. “smartphone”), and the Cloud, which allows for data transfer between multiple platforms and devices. Some devices comprised in the connected health care system include a toothbrush, a connected surface with sensors, a salivary diagnostic system, a breath analysis system, and further devices that will be explored in the subsequent sections. The connected health care system further provides a user identification system utilizing capacitive coupling of the human bo

In [136]:
util_patent[0]['citedby_patents']

[{'citedby_patent_number': '10076306', 'citedby_patent_date': '2018-09-18'},
 {'citedby_patent_number': '10201701', 'citedby_patent_date': '2019-02-12'},
 {'citedby_patent_number': '10869741', 'citedby_patent_date': '2020-12-22'}]

In [154]:
all_fp_list = []

for p in tqdm(util_patent):
    p_year = int(p['patent_date'].split('-')[0])
    fp_list = []
    
    for fp in p['citedby_patents']:
        try:
#             if fp['citedby_patent_date']
            fp_year = int(fp['citedby_patent_date'].split('-')[0])
            if p_year+5 >= fp_year:
                fp_list.append(fp['citedby_patent_number'])
            else:
                pass
        
        except AttributeError:
            pass
        except TypeError:
            pass
    
    all_fp_list.append(fp_list)

100%|█████████████████████████████████████████████████████████████████████████| 33217/33217 [00:01<00:00, 31031.81it/s]


In [155]:
all_fp_list_length = [len(p) for p in all_fp_list]
len(all_fp_list_length)

33217

In [156]:
all_fp_list_length[27638]

3

In [157]:
def AAFC(util_patent):
    all_fp_list = []

    for p in tqdm(util_patent):
        p_year = int(p['patent_date'].split('-')[0]) # 타겟 특허의 승인연도
        fp_list = []

        for fp in p['citedby_patents']:
            try:
    #             if fp['citedby_patent_date']
                fp_year = int(fp['citedby_patent_date'].split('-')[0]) # 타겟특허의 fc들의 승인연도
                # 타겟특허가 승인된지 5년 이내 forward citation 횟수
                if p_year+5 >= fp_year:
                    fp_list.append(fp['citedby_patent_number'])
                else:
                    pass
            # 오류 예외처리
            except AttributeError:
                pass
            except TypeError:
                pass

        all_fp_list.append(fp_list)
    # all_fp_list는 이중리스트의 형태이므로 각 원소리스트의 길이를 구하면서 flatten 진행
    all_fp_list_length = [len(p) for p in all_fp_list]
    return all_fp_list_length

In [159]:
aafc = AAFC(util_patent)

100%|█████████████████████████████████████████████████████████████████████████| 33217/33217 [00:00<00:00, 39010.29it/s]


In [161]:
final_patent_df = final_patent_df.drop(['AAFC'], axis=1)

In [163]:
final_patent_df['AAFC'] = aafc
final_patent_df

,patent_number,patent_title,patent_date,patent_type,patent_abstract,assignees,cited_patents,citedby_patents,examiners,cpc_set,...,grant_date,IKA,num_npl,examiner_impact,IKC,num_claims,under_examination_claim_index,heterogeneity,TCT,AAFC
0,9811636,Connected health care system,2017-11-07,utility,A connected health care system is provided wit...,[{'assignee_id': 'c6b8e6ee-b632-42c8-8f53-4d93...,"[{'cited_patent_number': None, 'cited_patent_d...","[{'citedby_patent_number': '10076306', 'citedb...","[{'examiner_id': '7djp2yknctv5h5pivqpgju4k2', ...","['A46B15', 'A46B15', 'A46B15', 'A46B15', 'A46B...",...,0.0,18,NaN,NaN,286,20,0.000000,0.687500,NaN,3
1,10064582,Noninvasive determination of cardiac health an...,2018-09-04,utility,This document describes assessment of human ph...,[{'assignee_id': '0bbbfb76-3528-467c-97dc-6d54...,"[{'cited_patent_number': '5410471', 'cited_pat...","[{'citedby_patent_number': '10376195', 'citedb...","[{'examiner_id': '3tpj24nwl1n2flyx4i5kqgeny', ...","['A47K13', 'A61B5', 'A61B5', 'A61B5', 'A61B5',...",...,NaN,2,1.0,NaN,228,18,0.000000,0.997582,3605.0,4
2,10827928,Real-time monitoring systems and methods in a ...,2020-11-10,utility,An apparatus for real time monitoring of a pat...,[{'assignee_id': '9251c0e2-7d30-4044-bf81-97a0...,"[{'cited_patent_number': '6374371', 'cited_pat...","[{'citedby_patent_number': None, 'citedby_pate...","[{'examiner_id': 'm7qdml3up2oslhidj79c0baom', ...","['A61B5', 'A61B5', 'A61B5', 'A61B5', 'A61B5', ...",...,NaN,5,NaN,NaN,247,31,10.333333,0.000000,3248.0,0
3,10052023,"Floor mat system and associated, computer medi...",2018-08-21,utility,"Provided are embodiments of systems, computer ...",[{'assignee_id': '827c2dfa-4040-4d6d-bc60-a8d4...,"[{'cited_patent_number': '4951963', 'cited_pat...","[{'citedby_patent_number': '10769574', 'citedb...","[{'examiner_id': '1ppzmlnxzqlegochzvy0mowbe', ...","['A61B5', 'A61B5', 'A61B5', 'A61B5', 'A61B5', ...",...,NaN,15,3.0,NaN,163,9,11.466667,0.995723,5908.0,1
4,10206625,"Chair pad system and associated, computer medi...",2019-02-19,utility,"Provided are embodiments of systems, computer ...",[{'assignee_id': '827c2dfa-4040-4d6d-bc60-a8d4...,"[{'cited_patent_number': '4951963', 'cited_pat...","[{'citedby_patent_number': '10769574', 'citedb...","[{'examiner_id': 'asxak897kt9a7cqtxv2syvv4m', ...","['A61B5', 'A61B5', 'A61B5', 'A61B5', 'A61B5', ...",...,NaN,16,6.0,NaN,0,NoClaimText,0.000000,0.995723,5908.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33212,10586614,System and method for timely multi-channel not...,2020-03-10,utility,A computer-assisted method to timely provide n...,[{'assignee_id': '3cb52395-6f90-4137-a8a4-0766...,"[{'cited_patent_number': None, 'cited_patent_d...","[{'citedby_patent_number': None, 'citedby_pate...","[{'examiner_id': '7fvqd8x22jc9k81zwryk9m3wr', ...","['G06N3', 'G06N20', 'G06N3', 'G06N5', 'G16H10'...",...,0.0,7,NaN,NaN,428,18,9.500000,0.583333,NaN,0
33213,10695902,Pharmacy order processing system,2020-06-30,utility,A pharmacy order processing system includes a ...,[{'assignee_id': 'ba0fcc10-50b2-446f-8811-bb06...,"[{'cited_patent_number': '4944647', 'cited_pat...","[{'citedby_patent_number': None, 'citedby_pate...","[{'examiner_id': 'xrvecakaq07dvhhoe9d7yy7z0', ...","['B25J9', 'B25J11', 'B25J9', 'B25J9', 'B25J9',...",...,NaN,20,NaN,NaN,404,30,0.000000,0.597701,4529.0,0
33214,10894027,Sickle cell disease treatment utilizing omega-...,2021-01-19,utility,Described herein are compositions including at...,[{'assignee_id': '63f3230c-69aa-4acc-9acb-a1bd...,"[{'cited_patent_number': '4513008', 'cited_pat...","[{'citedby_patent_number': None, 'citedby_pate...","[{'examiner_id': '3bxgj64hbifpvt5sxeowzom1x', ...","['A61K31', 'A61K9', 'A61K9', 'A61K31', 'A61K31...",...,NaN,10,NaN,NaN,0,NoClaimText,9.171196,0.666667,3409.0,0
33215,10643746,Image viewer,2020-05-05,utility,An image viewer and method for using the same ...,[{'assignee_id': '720a62eb-e056-40c7-88c9-176e...,"[{'cited_pat

In [164]:
final_patent_df['AAFC'].describe()

count    33217.000000
mean         3.140801
std         12.606624
min          0.000000
25%          0.000000
50%          0.000000
75%          2.000000
max        425.000000
Name: AAFC, dtype: float64

In [165]:
# Excel로 저장
final_patent_df.to_excel('./data/final_patent.xlsx', encoding='utf8', index=False)

In [ ]:
### originality 계산

# 특정 데이터 타입 모양의 문자열을 해당 모양의 데이터 타입으로 바꿔줌(ex. 리스트 모양 문자열을 진짜 리스트 타입으로)
from ast import literal_eval
for p in util_patent:
    try:
        p['forward_cpcs'] = literal_eval(p['forward_cpcs'])
    except SyntaxError:
        pass

def originality(util_patent):
    aa = []
    for p in tqdm(util_patent):
        a = p['forward_cpcs']
        aa.append(a)

    # aa = list(filter(None, aa))

    ### 추출한 forward citation의 CPC를 target patent끼리 합침
    gg = []
    for i in tqdm(aa):
        ggg = []
        for h in i:
            for j in forward_util_patent:
                try:
                    if h == j['patent_number']:
                        ggg.extend(j['forward_cpc_set'])
                except:
                    ggg.extend('')
        gg.append(ggg)

    ### 1-(HHI) 계산

    ### CPC 종류별 개수 구하기
    q = []
    for g in tqdm(gg):
        result = Counter(g)
        q.append(result)

    qq = []
    for a in q:
        b = list(a.values())
        qq.append(b)

    ### 구한 개수를 바탕으로 1-(HHI) 계산
    qqqq = []
    for r in tqdm(qq):
        a = sum(r)
        qqq = []
        for b in r:
            try:
                each_prop_square = (b/a)*(b/a)
                qqq.append(each_prop_square)
            except ZeroDivisionError:
                qqq.append(0)

        c = 1-sum(qqq)
        qqqq.append(c)

    ### target patent별로 originality 맵핑
    pats = []
    for n in tqdm(util_patent):
        pats.append(n['patent_number'])

    dic = {num:hhi for num, hhi in zip(pats, qqqq)}
    
    return dic

    
    
for a in tqdm(originality(util_patent)):
    a['originality'] = originality(util_patent)[a['patent_number']]

In [ ]:
### radicalness

def radicalness(util_patent, backward_util_patent):
    aa = []
    for p in tqdm(util_patent):
        a = p['backward_cpcs']
        aa.append(a)

    bb = []
    for i in tqdm(aa):
        bbb = []
        for h in i:
            for j in backward_util_patent:
                try:
                    if h == j['patent_number']:
                        bbb.extend(j['backward_cpc_set'])
                except:
                    bbb.extend('')
        bb.append(bbb)

    radicalness = []
    for i in tqdm(range(len(util_patent))):
        a = [c for c in bb[i] if c not in util_patent[i]['cpc_set']]
        try:
            radical = len(a)/len(bb[i])
        except ZeroDivisionError:
            radical = 'No BC'
        radicalness.append(radical)
    
    return radicalness

radi = radicalness(util_patent, backward_util_patent)
for i in tqdm(range(len(util_patent))):
    util_patent[i]['radicalness'] = radi[i]

In [ ]:
### broad impact

def broad_impact(util_patent, forward_util_patent):
    
    fc = []
    for p in tqdm(util_patent):
        a = p['forward_cpcs']
        fc.append(a)

    FCs = []
    for i in tqdm(fc):
        bbb = []
        for h in i:
            for j in forward_util_patent:
                try:
                    if h == j['patent_number']:
                        bbb.extend(j['forward_cpc_set'])
                except:
                    bbb.extend('')
        FCs.append(bbb)

    broad_impact = []
    for i in tqdm(range(len(util_patent))):
        a = [c for c in FCs[i] if c not in util_patent[i]['cpc_set']]
        try:
            impact = len(a)/len(FCs[i])
        except ZeroDivisionError:
            impact = 'No FC'
        broad_impact.append(impact)
    return broad_impact



bi = broad_impact(util_patent, forward_util_patent)

for i in tqdm(range(len(util_patent))):
    util_patent[i]['broad_impact'] = bi[i]

In [98]:
from ast import literal_eval
for p in util_patent:
    try:
        p['cited_patents'] = literal_eval(p['cited_patents'])
    except SyntaxError:
        pass

In [115]:
### TCT (median date of patent granted)
from datetime import datetime
    
for d in tqdm(util_patent):
    tct = []
    wrong_pat_nums = []
    for c in d['cited_patents']:
        try:
            if c['cited_patent_date'] is not None:
                grantdate_cited_patents = datetime.strptime(c['cited_patent_date'], '%Y-%m-%d') # 문자열 형태의 날짜를 실제 날짜 타입으로 변환
                time_diff = datetime.now() - grantdate_cited_patents
                tct.append(time_diff.days)
            else:
                pass
        except TypeError:
            wrong_pat_nums.append(d['patent_number'])
   
    d['TCT'] = np.median(tct)

100%|██████████████████████████████████████████████████████████████████████████| 33217/33217 [00:12<00:00, 2741.98it/s]


In [116]:
# dict to dataframe
final_patent_df = pd.DataFrame.from_dict(util_patent)
final_patent_df.drop_duplicates(['patent_abstract'], inplace=True)

In [117]:
# Excel로 저장
final_patent_df.to_excel('./data/final_patent.xlsx', encoding='utf8', index=False)

In [31]:
from ast import literal_eval
for p in util_patent:
    try:
        p['cited_patents'] = literal_eval(p['cited_patents'])
    except SyntaxError:
        pass

In [44]:
print(util_patent[1]['cited_patents'], '\n')

print(util_patent[1]['cited_patents'][0]['cited_patent_date'])

[{'cited_patent_number': '5410471', 'cited_patent_date': '1995-04-25'}, {'cited_patent_number': '5798798', 'cited_patent_date': '1998-08-25'}, {'cited_patent_number': '6254544', 'cited_patent_date': '2001-07-03'}, {'cited_patent_number': '6386757', 'cited_patent_date': '2002-05-14'}, {'cited_patent_number': '6513970', 'cited_patent_date': '2003-02-04'}, {'cited_patent_number': '6524239', 'cited_patent_date': '2003-02-25'}, {'cited_patent_number': '6616613', 'cited_patent_date': '2003-09-09'}, {'cited_patent_number': '7194371', 'cited_patent_date': '2007-03-20'}, {'cited_patent_number': '7421061', 'cited_patent_date': '2008-09-02'}, {'cited_patent_number': '7647093', 'cited_patent_date': '2010-01-12'}, {'cited_patent_number': '7677729', 'cited_patent_date': '2010-03-16'}, {'cited_patent_number': '7691067', 'cited_patent_date': '2010-04-06'}, {'cited_patent_number': '7698154', 'cited_patent_date': '2010-04-13'}, {'cited_patent_number': '8062220', 'cited_patent_date': '2011-11-22'}, {'cit

In [53]:
### examiner role

# 특정 데이터 타입 모양의 문자열을 해당 모양의 데이터 타입으로 바꿔줌(ex. 리스트 모양 문자열을 진짜 리스트 타입으로)

from ast import literal_eval
for p in util_patent:
    p['examiners'] = literal_eval(p['examiners'])

for d in util_patent:
    examiner=[]
    for c in d['examiners']:
        try:
            role = c['examiner_role']
            examiner.append(role)
        except:
            examiner.append('None')
    
    if 'assistant' in examiner:
        d['examiner_type'] = 1
    else:
        d['examiner_type'] = 0

In [8]:
### External knowledge
############################################ util patent 데이터프레임화 하고 저장하기 전에 해당 변수 코드 돌리기

# Importing necessary packages.
import os
import zipfile as zip
import pandas as pd
import csv
import numpy as np

# Set up file path:
# Please include the folder path of the file you are reading. Ex: os.chdir("C:/Users/johnsmith/Downloads")
# os.chdir("")

file_name = "./data/otherreference.tsv.zip"
f_name = "otherreference.tsv"
# Selecting the zip file.
zf = zip.ZipFile(file_name)
# Reading the selected file in the zip.
external_knowledge_df = pd.read_csv(zf.open(f_name), delimiter="\t", quoting = csv.QUOTE_NONNUMERIC)

# chunksize = 15*(10 ** 5)
# count = 1
# n_obs = 0
# dtype={'sequence': int}
# for external_knowledge_df in pd.read_csv(zf.open(f_name), delimiter="\t", chunksize=chunksize):
#     print('processing chunk: ' + str(count))
#     n_obs += len(external_knowledge_df)
#     count += 1

external_knowledge = pd.DataFrame(external_knowledge_df.groupby('patent_id').size(), columns=['num_npl'])
external_knowledge = external_knowledge.reset_index()
external_knowledge = external_knowledge.rename(columns={'patent_id':'patent_number'})

# 숫자인 행만 추출
text_deleted_external_knowledge = external_knowledge[external_knowledge['patent_number'].astype(str).str.isnumeric()]

df_ = pd.DataFrame.from_dict(util_patent)
df_['patent_number'] = df_['patent_number'].astype(int)
text_deleted_external_knowledge['patent_number'] = text_deleted_external_knowledge['patent_number'].astype(int)

# 특허 데이터프레임에 external knowledge를 vlookup적용
df_ = pd.merge(df_, text_deleted_external_knowledge, how='left', on='patent_number')
df_ = df_.drop(['num_npl_x'], axis=1)
df_ = df_.rename(columns={'num_npl_y':'num_npl'})

processing chunk: 1
processing chunk: 2
processing chunk: 3
processing chunk: 4
processing chunk: 5
processing chunk: 6
processing chunk: 7
processing chunk: 8
processing chunk: 9
processing chunk: 10
processing chunk: 11
processing chunk: 12
processing chunk: 13
processing chunk: 14
processing chunk: 15
processing chunk: 16
processing chunk: 17
processing chunk: 18
processing chunk: 19
processing chunk: 20
processing chunk: 21
processing chunk: 22
processing chunk: 23
processing chunk: 24
processing chunk: 25
processing chunk: 26
processing chunk: 27
processing chunk: 28
processing chunk: 29
processing chunk: 30
processing chunk: 31


In [ ]:
# ### Examiner impact

# file_name = "./data/uspatentcitation.tsv.zip"
# f_name = "uspatentcitation.tsv"
# zf = zip.ZipFile(file_name)
# examiner_impact_df_1 = pd.read_csv(zf.open(f_name), delimiter="\t", quoting = csv.QUOTE_NONNUMERIC)

# chunksize = 2*(10 ** 6)
# count = 1
# n_obs = 0
# for examiner_impact_df_1 in pd.read_csv(zf.open(f_name), delimiter="\t", chunksize=chunksize, quoting=csv.QUOTE_NONNUMERIC):
#     print('processing chunk: ' + str(count))
#     n_obs += len(examiner_impact_df_1)
#     count += 1
# # Print summary of data: number of observations, columns, and each variable data type
# # print(n_obs)
# # print(df.dtypes)



In [9]:
# Importing necessary packages.
import os
import zipfile as zip
import pandas as pd
import csv
# Set up file path:
# Please include the folder path of the file you are reading. Ex: os.chdir("C:/Users/johnsmith/Downloads")
# os.chdir("")
# Selecting the zip file.
file_name = "./data/uspatentcitation.tsv.zip"
f_name = "uspatentcitation.tsv"
zf = zip.ZipFile(file_name)
chunksize = 2*(10 ** 6)
count = 1
n_obs = 0
for examiner_impact_df_1 in pd.read_csv(zf.open(f_name), delimiter="\t", chunksize=chunksize, quoting=csv.QUOTE_NONNUMERIC):
    print('processing chunk: ' + str(count))
    n_obs += len(examiner_impact_df_1)
    count += 1
# # Print summary of data: number of observations, columns, and each variable data type
# print(n_obs)
# print(df.dtypes)

processing chunk: 1
processing chunk: 2
processing chunk: 3
processing chunk: 4
processing chunk: 5
processing chunk: 6
processing chunk: 7
processing chunk: 8
processing chunk: 9
processing chunk: 10
processing chunk: 11
processing chunk: 12
processing chunk: 13
processing chunk: 14
processing chunk: 15
processing chunk: 16
processing chunk: 17
processing chunk: 18
processing chunk: 19
processing chunk: 20
processing chunk: 21
processing chunk: 22
processing chunk: 23
processing chunk: 24
processing chunk: 25
processing chunk: 26
processing chunk: 27
processing chunk: 28
processing chunk: 29
processing chunk: 30
processing chunk: 31
processing chunk: 32
processing chunk: 33
processing chunk: 34
processing chunk: 35
processing chunk: 36
processing chunk: 37
processing chunk: 38
processing chunk: 39
processing chunk: 40
processing chunk: 41
processing chunk: 42
processing chunk: 43
processing chunk: 44
processing chunk: 45
processing chunk: 46
processing chunk: 47
processing chunk: 48
p

NameError: name 'df' is not defined

In [58]:
examiner_impact_df = examiner_impact_df_1.loc[examiner_impact_df_1['category'] == 'cited by examiner']

examiner_impact = pd.DataFrame(examiner_impact_df.groupby('patent_id').size(), columns=['examiner_impact'])
examiner_impact = examiner_impact.reset_index()
examiner_impact = examiner_impact.rename(columns={'patent_id':'patent_number'})

In [59]:
# 숫자인 행만 추출
text_deleted_examiner_impact = examiner_impact[examiner_impact['patent_number'].astype(str).str.isnumeric()]
text_deleted_examiner_impact

,patent_number,examiner_impact
0,10000027,1
1,10000122,1
2,10000176,1
3,10000179,1
4,10000270,1
...,...,...
181751,9999840,1
181752,9999892,1
181753,9999954,1
181754,9999981,1


In [60]:
text_deleted_examiner_impact['examiner_impact'].describe()

count    181756.000000
mean          1.058380
std           0.260707
min           1.000000
25%           1.000000
50%           1.000000
75%           1.000000
max           9.000000
Name: examiner_impact, dtype: float64

In [26]:
final_patent_df['patent_number'] = final_patent_df['patent_number'].astype(int)
text_deleted_examiner_impact['patent_number'] = text_deleted_examiner_impact['patent_number'].astype(int)

C:\Users\Woojin\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [27]:
df_ = pd.merge(final_patent_df, text_deleted_examiner_impact, how='left', on='patent_number')

In [28]:
df_

,patent_number,patent_title,patent_date,patent_type,patent_abstract,assignees,cited_patents,citedby_patents,examiners,cpc_set,...,forward_cpcs,AAFC,backward_cpcs,originality,radicalness,broad_impact,grant_date,IKA,num_npl,examiner_impact
0,9811636,Connected health care system,2017-11-07,utility,A connected health care system is provided wit...,[{'assignee_id': 'c6b8e6ee-b632-42c8-8f53-4d93...,"[{'cited_patent_number': None, 'cited_patent_d...","[{'citedby_patent_number': '10076306', 'citedb...","[{'examiner_id': '7djp2yknctv5h5pivqpgju4k2', ...","['A46B15', 'A46B15', 'A46B15', 'A46B15', 'A46B...",...,"['10076306', '10201701', '10869741']",0.750000,[],0.807889,No BC,0.351351,0.0,0,NaN,NaN
1,10064582,Noninvasive determination of cardiac health an...,2018-09-04,utility,This document describes assessment of human ph...,[{'assignee_id': '0bbbfb76-3528-467c-97dc-6d54...,"[{'cited_patent_number': '5410471', 'cited_pat...","[{'citedby_patent_number': '10376195', 'citedb...","[{'examiner_id': '3tpj24nwl1n2flyx4i5kqgeny', ...","['A47K13', 'A61B5', 'A61B5', 'A61B5', 'A61B5',...",...,"['10376195', '10936081', '10936085', '10948996']",1.333333,"['5410471', '5798798', '6254544', '6386757', '...",0.798334,0.353086,0.634146,NaN,6,1.0,NaN
2,10827928,Real-time monitoring systems and methods in a ...,2020-11-10,utility,An apparatus for real time monitoring of a pat...,[{'assignee_id': '9251c0e2-7d30-4044-bf81-97a0...,"[{'cited_patent_number': '6374371', 'cited_pat...","[{'citedby_patent_number': None, 'citedby_pate...","[{'examiner_id': 'm7qdml3up2oslhidj79c0baom', ...","['A61B5', 'A61B5', 'A61B5', 'A61B5', 'A61B5', ...",...,[],0,"['6374371', '6882883', '7229409', '8002701', '...",1.000000,0.295082,No FC,NaN,4,NaN,NaN
3,10052023,"Floor mat system and associated, computer medi...",2018-08-21,utility,"Provided are embodiments of systems, computer ...",[{'assignee_id': '827c2dfa-4040-4d6d-bc60-a8d4...,"[{'cited_patent_number': '4951963', 'cited_pat...","[{'citedby_patent_number': '10769574', 'citedb...","[{'examiner_id': '1ppzmlnxzqlegochzvy0mowbe', ...","['A61B5', 'A61B5', 'A61B5', 'A61B5', 'A61B5', ...",...,['10769574'],0.333333,"['4951963', '4998534', '5000188', '5111539', '...",0.000000,0.566337,1,NaN,2,3.0,NaN
4,10206625,"Chair pad system and associated, computer medi...",2019-02-19,utility,"Provided are embodiments of systems, computer ...",[{'assignee_id': '827c2dfa-4040-4d6d-bc60-a8d4...,"[{'cited_patent_number': '4951963', 'cited_pat...","[{'citedby_patent_number': '10769574', 'citedb...","[{'examiner_id': 'asxak897kt9a7cqtxv2syvv4m', ...","['A61B5', 'A61B5', 'A61B5', 'A61B5', 'A61B5', ...",...,['10769574'],0.500000,"['4951963', '4998534', '5000188', '5111539', '...",0.000000,0.571287,1,NaN,4,6.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33212,10586614,System and method for timely multi-channel not...,2020-03-10,utility,A computer-assisted method to timely provide n...,[{'assignee_id': '3cb52395-6f90-4137-a8a4-0766...,"[{'cited_patent_number': None, 'cited_patent_d...","[{'citedby_patent_number': None, 'citedby_pate...","[{'examiner_id': '7fvqd8x22jc9k81zwryk9m3wr', ...","['G06N3', 'G06N20', 'G06N3', 'G06N5', 'G16H10'...",...,[],0,[],1.000000,No BC,No FC,0.0,7,NaN,NaN
33213,10695902,Pharmacy order processing system,2020-06-30,utility,A pharmacy order processing system includes a ...,[{'assignee_id': 'ba0fcc10-50b2-446f-8811-bb06...,"[{'cited_patent_number': '4944647', 'cited_pat...","[{'citedby_patent_number': None, 'citedby_pate...","[{'examiner_id': 'xrvecakaq07dvhhoe9d7yy7z0', ...","['B25J9', 'B25J11', 'B25J9', 'B25J9', 'B25J9',...",...,[],0,"['4944647', '5720154', '5771657', '6026561', '...",1.000000,0.910180,No FC,NaN,20,NaN,NaN
33214,10894027,Sickle cell disease treatment utilizing omega-...,2021-01-19,utility,Described herein are compositions including at...,[{'assignee_id': '63f3230c-69aa-4acc-9acb-a1bd...,"[{'cited_patent_number': '4513008', 'cited_pat...","[{'citedby_patent_number

In [29]:
df_['examiner_impact'].describe()

count    843.000000
mean       1.035587
std        0.215030
min        1.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        3.000000
Name: examiner_impact, dtype: float64

In [30]:
# CSV로 저장
df_.to_csv('./data/final_patent.csv', encoding='utf8', index=False)

In [31]:
# Excel로 저장
df_.to_excel('./data/final_patent.xlsx', encoding='utf8', index=False)

In [61]:
xxx = pd.DataFrame.from_dict(util_patent)
xxx

,patent_number,patent_title,patent_date,patent_type,patent_abstract,assignees,cited_patents,citedby_patents,examiners,cpc_set,...,examiner_type,forward_cpcs,AAFC,backward_cpcs,originality,radicalness,broad_impact,grant_date,IKA,num_npl
0,9811636,Connected health care system,2017-11-07,utility,A connected health care system is provided wit...,[{'assignee_id': 'c6b8e6ee-b632-42c8-8f53-4d93...,"[{'cited_patent_number': None, 'cited_patent_d...","[{'citedby_patent_number': '10076306', 'citedb...","[{'examiner_id': '7djp2yknctv5h5pivqpgju4k2', ...","['A46B15', 'A46B15', 'A46B15', 'A46B15', 'A46B...",...,0,"['10076306', '10201701', '10869741']",0.750000,[],0.807889,No BC,0.351351,0.0,0,NaN
1,10064582,Noninvasive determination of cardiac health an...,2018-09-04,utility,This document describes assessment of human ph...,[{'assignee_id': '0bbbfb76-3528-467c-97dc-6d54...,"[{'cited_patent_number': '5410471', 'cited_pat...","[{'citedby_patent_number': '10376195', 'citedb...","[{'examiner_id': '3tpj24nwl1n2flyx4i5kqgeny', ...","['A47K13', 'A61B5', 'A61B5', 'A61B5', 'A61B5',...",...,0,"['10376195', '10936081', '10936085', '10948996']",1.333333,"['5410471', '5798798', '6254544', '6386757', '...",0.798334,0.353086,0.634146,NaN,6,1.0
2,10827928,Real-time monitoring systems and methods in a ...,2020-11-10,utility,An apparatus for real time monitoring of a pat...,[{'assignee_id': '9251c0e2-7d30-4044-bf81-97a0...,"[{'cited_patent_number': '6374371', 'cited_pat...","[{'citedby_patent_number': None, 'citedby_pate...","[{'examiner_id': 'm7qdml3up2oslhidj79c0baom', ...","['A61B5', 'A61B5', 'A61B5', 'A61B5', 'A61B5', ...",...,1,[],0,"['6374371', '6882883', '7229409', '8002701', '...",1.000000,0.295082,No FC,NaN,4,NaN
3,10052023,"Floor mat system and associated, computer medi...",2018-08-21,utility,"Provided are embodiments of systems, computer ...",[{'assignee_id': '827c2dfa-4040-4d6d-bc60-a8d4...,"[{'cited_patent_number': '4951963', 'cited_pat...","[{'citedby_patent_number': '10769574', 'citedb...","[{'examiner_id': '1ppzmlnxzqlegochzvy0mowbe', ...","['A61B5', 'A61B5', 'A61B5', 'A61B5', 'A61B5', ...",...,1,['10769574'],0.333333,"['4951963', '4998534', '5000188', '5111539', '...",0.000000,0.566337,1,NaN,2,3.0
4,10206625,"Chair pad system and associated, computer medi...",2019-02-19,utility,"Provided are embodiments of systems, computer ...",[{'assignee_id': '827c2dfa-4040-4d6d-bc60-a8d4...,"[{'cited_patent_number': '4951963', 'cited_pat...","[{'citedby_patent_number': '10769574', 'citedb...","[{'examiner_id': 'asxak897kt9a7cqtxv2syvv4m', ...","['A61B5', 'A61B5', 'A61B5', 'A61B5', 'A61B5', ...",...,1,['10769574'],0.500000,"['4951963', '4998534', '5000188', '5111539', '...",0.000000,0.571287,1,NaN,4,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33212,10586614,System and method for timely multi-channel not...,2020-03-10,utility,A computer-assisted method to timely provide n...,[{'assignee_id': '3cb52395-6f90-4137-a8a4-0766...,"[{'cited_patent_number': None, 'cited_patent_d...","[{'citedby_patent_number': None, 'citedby_pate...","[{'examiner_id': '7fvqd8x22jc9k81zwryk9m3wr', ...","['G06N3', 'G06N20', 'G06N3', 'G06N5', 'G16H10'...",...,1,[],0,[],1.000000,No BC,No FC,0.0,7,NaN
33213,10695902,Pharmacy order processing system,2020-06-30,utility,A pharmacy order processing system includes a ...,[{'assignee_id': 'ba0fcc10-50b2-446f-8811-bb06...,"[{'cited_patent_number': '4944647', 'cited_pat...","[{'citedby_patent_number': None, 'citedby_pate...","[{'examiner_id': 'xrvecakaq07dvhhoe9d7yy7z0', ...","['B25J9', 'B25J11', 'B25J9', 'B25J9', 'B25J9',...",...,0,[],0,"['4944647', '5720154', '5771657', '6026561', '...",1.000000,0.910180,No FC,NaN,20,NaN
33214,10894027,Sickle cell disease treatment utilizing omega-...,2021-01-19,utility,Described herein are compositions including at...,[{'assignee_id': '63f3230c-69aa-4acc-9acb-a1bd...,"[{'cited_patent_number': '4513008', 'cited_pat...","[{'citedby_patent_number': None, 'citedb

In [62]:
xxx['examiner_type'].describe()

count    33217.000000
mean         0.283198
std          0.450559
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: examiner_type, dtype: float64

In [63]:
# Excel로 저장
xxx.to_excel('./data/examiner_type.xlsx', encoding='utf8', index=False)

In [43]:
import csv, sys, ctypes as ct

limit0 = csv.field_size_limit()
limit0

131072

In [44]:
sys.platform, sys.maxsize, ct.sizeof(ct.c_void_p) * 8, ct.sizeof(ct.c_long) * 8

csv.field_size_limit(int(ct.c_ulong(-1).value // 2))
limit1 = csv.field_size_limit()
limit1

2147483647

In [45]:
### Claim
import os
import zipfile as zip
import pandas as pd
import csv
import numpy as np
# Set up file path:
# Please include the folder path of the file you are reading. Ex: os.chdir("C:/Users/johnsmith/Downloads")
# os.chdir("")

file_name_2002 = "./data/claims_2002.tsv.zip"
f_name_2002 = "claims_2002.tsv"
# Selecting the zip file.
zf = zip.ZipFile(file_name_2002)
# Reading the selected file in the zip.
df_2002 = pd.read_csv(zf.open(f_name_2002), usecols=['patent_id', 'text'], delimiter="\t", quoting=csv.QUOTE_NONNUMERIC)

file_name_2003 = "./data/claims_2003.tsv.zip"
f_name_2003 = "claims_2003.tsv"
# Selecting the zip file.
zf = zip.ZipFile(file_name_2003)
# Reading the selected file in the zip.
df_2003 = pd.read_csv(zf.open(f_name_2003), usecols=['patent_id', 'text'], delimiter="\t", quoting=csv.QUOTE_NONNUMERIC)

file_name_2004 = "./data/claims_2004.tsv.zip"
f_name_2004 = "claims_2004.tsv"
# Selecting the zip file.
zf = zip.ZipFile(file_name_2004)
# Reading the selected file in the zip.
df_2004 = pd.read_csv(zf.open(f_name_2004), usecols=['patent_id', 'text'], delimiter="\t", quoting=csv.QUOTE_NONNUMERIC)

file_name_2005 = "./data/claims_2005.tsv.zip"
f_name_2005 = "claims_2005.tsv"
# Selecting the zip file.
zf = zip.ZipFile(file_name_2005)
# Reading the selected file in the zip.
df_2005 = pd.read_csv(zf.open(f_name_2005), usecols=['patent_id', 'text'], delimiter="\t", quoting=csv.QUOTE_NONNUMERIC)

file_name_2006 = "./data/claims_2006.tsv.zip"
f_name_2006 = "claims_2006.tsv"
# Selecting the zip file.
zf = zip.ZipFile(file_name_2006)
# Reading the selected file in the zip.
df_2006 = pd.read_csv(zf.open(f_name_2006), usecols=['patent_id', 'text'], delimiter="\t", quoting=csv.QUOTE_NONNUMERIC)

file_name_2007 = "./data/claims_2007.tsv.zip"
f_name_2007 = "claims_2007.tsv"
# Selecting the zip file.
zf = zip.ZipFile(file_name_2007)
# Reading the selected file in the zip.
df_2007 = pd.read_csv(zf.open(f_name_2007), usecols=['patent_id', 'text'], delimiter="\t", quoting=csv.QUOTE_NONNUMERIC)

file_name_2008 = "./data/claims_2008.tsv.zip"
f_name_2008 = "claims_2008.tsv"
# Selecting the zip file.
zf = zip.ZipFile(file_name_2008)
# Reading the selected file in the zip.
df_2008 = pd.read_csv(zf.open(f_name_2008), usecols=['patent_id', 'text'], delimiter="\t", quoting=csv.QUOTE_NONNUMERIC)

file_name_2009 = "./data/claims_2009.tsv.zip"
f_name_2009 = "claims_2009.tsv"
# Selecting the zip file.
zf = zip.ZipFile(file_name_2009)
# Reading the selected file in the zip.
df_2009 = pd.read_csv(zf.open(f_name_2009), usecols=['patent_id', 'text'], delimiter="\t", quoting=csv.QUOTE_NONNUMERIC)

file_name_2010 = "./data/claims_2010.tsv.zip"
f_name_2010 = "claims_2010.tsv"
# Selecting the zip file.
zf = zip.ZipFile(file_name_2010)
# Reading the selected file in the zip.
df_2010 = pd.read_csv(zf.open(f_name_2010), usecols=['patent_id', 'text'], delimiter="\t", quoting=csv.QUOTE_NONNUMERIC)

file_name_2011 = "./data/claims_2011.tsv.zip"
f_name_2011 = "claims_2011.tsv"
# Selecting the zip file.
zf = zip.ZipFile(file_name_2011)
# Reading the selected file in the zip.
df_2011 = pd.read_csv(zf.open(f_name_2011), usecols=['patent_id', 'text'], delimiter="\t", quoting=csv.QUOTE_NONNUMERIC)

file_name_2012 = "./data/claims_2012.tsv.zip"
f_name_2012 = "claims_2012.tsv"
# Selecting the zip file.
zf = zip.ZipFile(file_name_2012)
# Reading the selected file in the zip.
df_2012 = pd.read_csv(zf.open(f_name_2012), usecols=['patent_id', 'text'], delimiter="\t", quoting=csv.QUOTE_NONNUMERIC)

file_name_2013 = "./data/claims_2013.tsv.zip"
f_name_2013 = "claims_2013.tsv"
# Selecting the zip file.
zf = zip.ZipFile(file_name_2013)
# Reading the selected file in the zip.
df_2013 = pd.read_csv(zf.open(f_name_2013), usecols=['patent_id', 'text'], delimiter="\t", quoting=csv.QUOTE_NONNUMERIC)

file_name_2014 = "./data/claims_2014.tsv.zip"
f_name_2014 = "claims_2014.tsv"
# Selecting the zip file.
zf = zip.ZipFile(file_name_2014)
# Reading the selected file in the zip.
df_2014 = pd.read_csv(zf.open(f_name_2014), usecols=['patent_id', 'text'], delimiter="\t", quoting=csv.QUOTE_NONNUMERIC)

file_name_2015 = "./data/claims_2015.tsv.zip"
f_name_2015 = "claims_2015.tsv"
# Selecting the zip file.
zf = zip.ZipFile(file_name_2015)
# Reading the selected file in the zip.
df_2015 = pd.read_csv(zf.open(f_name_2015), usecols=['patent_id', 'text'], delimiter="\t", quoting=csv.QUOTE_NONNUMERIC)

file_name_2016 = "./data/claims_2016.tsv.zip"
f_name_2016 = "claims_2016.tsv"
# Selecting the zip file.
zf = zip.ZipFile(file_name_2016)
# Reading the selected file in the zip.
df_2016 = pd.read_csv(zf.open(f_name_2016), usecols=['patent_id', 'text'], delimiter="\t", quoting=csv.QUOTE_NONNUMERIC)

file_name_2017 = "./data/claims_2017.tsv.zip"
f_name_2017 = "claims_2017.tsv"
# Selecting the zip file.
zf = zip.ZipFile(file_name_2017)
# Reading the selected file in the zip.
df_2017 = pd.read_csv(zf.open(f_name_2017), usecols=['patent_id', 'text'], delimiter="\t", quoting=csv.QUOTE_NONNUMERIC)

file_name_2018 = "./data/claims_2018.tsv.zip"
f_name_2018 = "claims_2018.tsv"
# Selecting the zip file.
zf = zip.ZipFile(file_name_2018)
# Reading the selected file in the zip.
df_2018 = pd.read_csv(zf.open(f_name_2018), usecols=['patent_id', 'text'], delimiter="\t", quoting=csv.QUOTE_NONNUMERIC)

file_name_2019 = "./data/claims_2019.tsv.zip"
f_name_2019 = "claims_2019.tsv"
# Selecting the zip file.
zf = zip.ZipFile(file_name_2019)
# Reading the selected file in the zip.
df_2019 = pd.read_csv(zf.open(f_name_2019), usecols=['patent_id', 'text'], delimiter="\t", quoting=csv.QUOTE_NONNUMERIC)

file_name_2020 = "./data/claims_2020.tsv.zip"
f_name_2020 = "claim_2020.tsv"
# Selecting the zip file.
zf = zip.ZipFile(file_name_2020)
# Reading the selected file in the zip.
df_2020 = pd.read_csv(zf.open(f_name_2020), usecols=['patent_id', 'text'], delimiter="\t", quoting=csv.QUOTE_NONNUMERIC)

file_name_2021 = "./data/claims_2021.tsv.zip"
f_name_2021 = "claims_2021.tsv"
# Selecting the zip file.
zf = zip.ZipFile(file_name_2021)
# Reading the selected file in the zip.
df_2021 = pd.read_csv(zf.open(f_name_2021), usecols=['patent_id', 'text'], delimiter="\t", quoting=csv.QUOTE_NONNUMERIC)

C:\Users\Woojin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


MemoryError: Unable to allocate 512. KiB for an array with shape (65536,) and data type float64

In [ ]:
total_period_claims = pd.concat([df_2002, df_2003, df_2004, df_2005, df_2006, df_2007, df_2008, df_2009, df_2010, df_2011, df_2012, df_2013, df_2014, df_2015, df_2016, df_2017, df_2018, df_2020, df_2021], axis=0)

# period_1_claims = pd.concat([df_2002, df_2003, df_2004, df_2005, df_2006, df_2007, df_2008], axis=0)
# period_2_claims = pd.concat([df_2009, df_2010, df_2011, df_2012, df_2013, df_2014, df_2015], axis=0)
# period_3_claims = pd.concat([df_2016, df_2017, df_2018, df_2020, df_2021], axis=0)


In [ ]:
df_ = pd.DataFrame(df.groupby('patent_id').size(), columns=['num_claims'])
df_ = df_.reset_index()
df_ = df_.rename(columns={'patent_id':'patent_number'})

In [63]:
def get_claim_list(claim_df_per_period):
    period_claims = claim_df_per_period.astype({'text':'str'})
    period_claims = period_claims.reset_index()
    # 각 특허별로 claim text 합치기
    period_claims__ = period_claims.groupby(['patent_id'])['text'].apply(lambda x: ''.join(set(x)))
    
    period_claims__ = period_claims__.reset_index()
    period_claims__ = period_claims__.rename(columns={'patent_id':'patent_number'})
    period_claims__ = period_claims__.set_index('patent_number')
    
    claim_list = []
    for p in util_patent:
        try:
            claim_list.append(period_claims__.loc[p['patent_number'], :].text)
        except KeyError:
            claim_list.append('NoClaimText')
    
    return claim_list

In [ ]:
final_patent_df['patent_claim'] = claim_list

In [ ]:
final_patent_df

In [ ]:
final_patent_df[['patent_number', 'patent_date', 'patent_claim']].to_excel('./data/기간2_claim.xlsx', encoding='utf8', index=False)

In [8]:
### irreplaceable keyword in claim

### 특허별로 기간 나누고 claim 추출
period_1_claim = []
period_2_claim = []
period_3_claim = []

for y in util_patent:
    yy = y['patent_date'].split('-')[0]
    if int(yy)<=2008:
        period_1_claim.append(y['patent_claim'])
        
    elif int(yy)>=2016:
        period_3_claim.append(y['patent_claim'])
        
    else:
        period_2_claim.append(y['patent_claim'])

In [368]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = stopwords.words('english')
stop_words.append('NoClaimText')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Woojin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Woojin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Woojin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [363]:
def tokenize(text):
    
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
   
    # tokenize text
    tokens = word_tokenize(text)
   
    # lemmatize and remove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
   
    return tokens

def IKA(abstract_list):
    cleaned_abstract = []
    for a in abstract_list:
        t = tokenize(a)
        tt = ' '.join(t)
        cleaned_abstract.append(tt)

    ### term X term matrix 생성

    tfidfv = TfidfVectorizer().fit(cleaned_abstract)
    doc_term_matrix = tfidfv.transform(cleaned_abstract)

    # column_name = []
    # for p in util_patent:
    #     column_name.append(p['patent_number'])

    # matrix = pd.DataFrame(doc_term_matrix.T.toarray(), columns=column_name, index=tfidfv.get_feature_names())
    # matrix

    period_vocab = dict()

    for idx, word in enumerate(tfidfv.get_feature_names()):
        period_vocab[word] = doc_term_matrix.getcol(idx).sum() ## number of patent X number of words의 형태에서 특허별로 특정 단어 tfidf 값 더함

    top10_words = sorted(period_vocab.items(), key=lambda x: x[1], reverse=True)[:10]

    abstract_keyword_list = [w[0] for w in top10_words]

    period_num_words = []
    for abstract in cleaned_abstract:
        num_k = []
        for keyword in abstract_keyword_list:
            num_k.append(abstract.count(keyword))
            total_num_k = sum(num_k)

        period_num_words.append(total_num_k)

    return period_num_words

def IKC(claim_list):
    
    cleaned_claim = []
    for a in claim_list:
        t = tokenize(a)
        tt = ' '.join(t)
        cleaned_claim.append(tt)

    ### term X term matrix 생성

    tfidfv = TfidfVectorizer().fit(cleaned_claim)
    doc_term_matrix = tfidfv.transform(cleaned_claim)

    # column_name = []
    # for p in util_patent:
    #     column_name.append(p['patent_number'])

    # matrix = pd.DataFrame(doc_term_matrix.T.toarray(), columns=column_name, index=tfidfv.get_feature_names())
    # matrix

    period_vocab = dict()

    for idx, word in enumerate(tfidfv.get_feature_names()):
        period_vocab[word] = doc_term_matrix.getcol(idx).sum() ## number of patent X number of words의 형태에서 특허별로 특정 단어 tfidf 값 더함

    top30_words = sorted(period_vocab.items(), key=lambda x: x[1], reverse=True)[:30]

    claim_keyword_list = [w[0] for w in top30_words]

    period_num_words = []
    for claim in cleaned_claim:
        num_k = []
        for keyword in claim_keyword_list:
            num_k.append(claim.count(keyword))
            total_num_k = sum(num_k)

        period_num_words.append(total_num_k)

    return period_num_words



In [11]:
period_1_num_claim_keywords = IKC(period_1_claim)
period_2_num_claim_keywords = IKC(period_2_claim)
period_3_num_claim_keywords = IKC(period_3_claim)

  0%|                                                                                        | 0/10832 [00:00<?, ?it/s]

  0%|                                                                                        | 0/20887 [00:00<?, ?it/s]

NameError: name 'whole_period_num_claim_keywords' is not defined

In [12]:
whole_period_num_claim_keywords=[]
whole_period_num_claim_keywords.extend(period_1_num_claim_keywords)
whole_period_num_claim_keywords.extend(period_2_num_claim_keywords)
whole_period_num_claim_keywords.extend(period_3_num_claim_keywords)

100%|██████████████████████████████████████████████████████████████████████████████| 1498/1498 [05:50<00:00,  4.27it/s]

100%|████████████████████████████████████████████████████████████████████████████| 10832/10832 [04:42<00:00, 38.32it/s]


100%|███████████████████████████████████████████████████████████████████████████| 20887/20887 [01:07<00:00, 309.19it/s]


In [20]:
period_1_claim_df = pd.read_excel('./data/only_기간1_claim.xlsx')
period_2_claim_df = pd.read_excel('./data/only_기간2_claim.xlsx')
period_3_claim_df = pd.read_excel('./data/only_기간3_claim.xlsx')

In [19]:
period_1_claim_df

,patent_number,patent_title,patent_date,patent_claim
0,6003240,Vessel drier having high heat efficiency,1999-12-21,NoClaimText
1,7260430,Architecture of an embedded internet robot sys...,2007-08-21,10. The architecture of an embedded internet r...
2,6972261,Method for fabricating fine features by jet-pr...,2005-12-06,NoClaimText
3,7163395,Dental measurement instruments,2007-01-16,NoClaimText
4,7470652,Cleaning composition for disposable cleaning h...,2008-12-30,"8. The cleaning implement of claim 5 , wherein..."
...,...,...,...,...
1493,7361747,Isolation and characterization of the precurso...,2008-04-22,"2. The nucleic acid molecule of claim 1 , wher..."
1494,7000963,Disposable holder for use with urine specimen cup,2006-02-21,NoClaimText
1495,6763088,Medical dispatch system,2004-07-13,"20. The method of claim 14 , wherein if the cu..."
1496,7398948,Respiratory disease management device and rela...,2008-07-15,NoClaimText


In [21]:
period_1_claim = list(period_1_claim_df['patent_claim'])
period_2_claim = list(period_2_claim_df['patent_claim'])
period_3_claim = list(period_3_claim_df['patent_claim'])

In [24]:
period_1_num_claim_keywords = IKC(period_1_claim)
period_2_num_claim_keywords = IKC(period_2_claim)
period_3_num_claim_keywords = IKC(period_3_claim)

In [25]:
period_1_claim_df['IKC'] = period_1_num_claim_keywords
period_2_claim_df['IKC'] = period_2_num_claim_keywords
period_3_claim_df['IKC'] = period_3_num_claim_keywords

In [26]:
period_1_claim_df.to_excel('./data/period_1_IKC.xlsx', encoding='utf8', index=False)
period_2_claim_df.to_excel('./data/period_2_IKC.xlsx', encoding='utf8', index=False)
period_3_claim_df.to_excel('./data/period_3_IKC.xlsx', encoding='utf8', index=False)

In [33]:
period_1_abstract_df = pd.read_excel('./data/only_기간1_abstract.xlsx')
period_2_abstract_df = pd.read_excel('./data/only_기간2_abstract.xlsx')
period_3_abstract_df = pd.read_excel('./data/only_기간3_abstract.xlsx')

In [34]:
period_1_abstract = list(period_1_abstract_df['patent_abstract'])
period_2_abstract = list(period_2_abstract_df['patent_abstract'])
period_3_abstract = list(period_3_abstract_df['patent_abstract'])

In [35]:
print(len(period_1_abstract)+len(period_2_abstract)+len(period_3_abstract))

33217


In [36]:
period_1_num_abstract_keywords = IKA(period_1_abstract)
period_2_num_abstract_keywords = IKA(period_2_abstract)
period_3_num_abstract_keywords = IKA(period_3_abstract)

In [37]:
period_1_abstract_df['IKA'] = period_1_num_abstract_keywords
period_2_abstract_df['IKA'] = period_2_num_abstract_keywords
period_3_abstract_df['IKA'] = period_3_num_abstract_keywords

In [38]:
period_1_abstract_df.to_excel('./data/period_1_IKA.xlsx', encoding='utf8', index=False)
period_2_abstract_df.to_excel('./data/period_2_IKA.xlsx', encoding='utf8', index=False)
period_3_abstract_df.to_excel('./data/period_3_IKA.xlsx', encoding='utf8', index=False)

In [39]:
final_patent_df = pd.read_excel('./data/final_patent.xlsx')

In [40]:
util_patent = final_patent_df.to_dict('records')

In [5]:
### under examination claim index

util_patent[0]

{'patent_number': 9811636,
 'patent_title': 'Connected health care system',
 'patent_date': '2017-11-07',
 'patent_type': 'utility',
 'patent_abstract': 'A connected health care system is provided with various devices for tracking and monitoring health statistics and behaviors including oral health, fitness, heart health, bone health, salivary diagnostics, diabetes, and further options that will be explored in the subsequent sections. The connected health care system is comprised of various devices, a data transfer medium (i.e. “smartphone”), and the Cloud, which allows for data transfer between multiple platforms and devices. Some devices comprised in the connected health care system include a toothbrush, a connected surface with sensors, a salivary diagnostic system, a breath analysis system, and further devices that will be explored in the subsequent sections. The connected health care system further provides a user identification system utilizing capacitive coupling of the human bo

In [11]:
import ast
final_patent_df['cpc_set'] = final_patent_df['cpc_set'].map(lambda x: ast.literal_eval(x))
util_patent = final_patent_df.to_dict('records')

In [14]:
type(util_patent[0]['num_claims'])

int

In [15]:
print(set(util_patent[0]['cpc_set']))

{'G16H40', 'A61C17', 'A61C19', 'G06Q40', 'G06Q10', 'A46B15', 'A46B2200', 'A61B5', 'G16H50', 'A46B5'}


In [58]:
# 모든 특허의 cpc set
whole_cpc_set=[]
for fp in util_patent:
    c = list(dict.fromkeys(fp['cpc_set'])) # 순서 유지하면서 중복 제거
    c = {v for v in c if v} # None 제거하고 속도를 위해 set로 저장
    whole_cpc_set.append(c)

In [59]:
util_patent[0]['cpc_set']

['A46B15',
 'A46B15',
 'A46B15',
 'A46B15',
 'A46B5',
 'A46B2200',
 'A61C19',
 'A61B5',
 'A61B5',
 'A61B5',
 'A61B5',
 'A61B5',
 'A61C17',
 'A61C17',
 'A61B5',
 'A61C17',
 'G06Q10',
 'G06Q40',
 'G16H40',
 'G16H50']

In [60]:
whole_cpc_set[0]

{'A46B15',
 'A46B2200',
 'A46B5',
 'A61B5',
 'A61C17',
 'A61C19',
 'G06Q10',
 'G06Q40',
 'G16H40',
 'G16H50'}

In [25]:
len(whole_cpc_set[0])

10

In [20]:
len(whole_cpc_set)

33217

In [44]:
len(b)

33217

In [85]:
for i, cpc_b in enumerate(whole_cpc_set[:5]):
    print(cpc_b)

{'G16H40', 'A61C17', 'A61C19', 'G06Q40', 'G06Q10', 'A46B15', 'A46B2200', 'A61B5', 'G16H50', 'A46B5'}
{'A61B2505', 'A61B2562', 'G16H40', 'A47K13', 'G01K11', 'G16H50', 'A61B8', 'A61B5', 'G16H15'}
{'G16Z99', 'G16H40', 'G06F19', 'G16H50', 'A61B5'}
{'G06F19', 'A61B5', 'G16H15', 'G16H40'}
{'A61B5', 'F04C2270', 'G16H15', 'G16H40'}


In [118]:
## 특정 특허와 0.9이상 비슷한 특허 인덱스 뽑기
similar_pat_list = []

for cpc_a in tqdm(whole_cpc_set):
    pat_num_index=[]
    
    for i, cpc_b in enumerate(whole_cpc_set):
        if len(cpc_a & cpc_b)/len(cpc_a) >= 0.9:
            pat_num_index.append(i)
        else:
            pass
    similar_pat_list.append(pat_num_index)

# similar_pat_list = list(itertools.chain(*similar_pat_list))

100%|████████████████████████████████████████████████████████████████████████████| 33217/33217 [06:10<00:00, 89.63it/s]


In [119]:
similar_pat_list

[[0],
 [1],
 [2, 98, 1617, 2656],
 [3,
  98,
  106,
  468,
  952,
  1051,
  2326,
  2656,
  16083,
  16111,
  16429,
  17299,
  19978,
  21279,
  22176,
  32796],
 [4],
 [5],
 [2,
  6,
  98,
  100,
  106,
  169,
  197,
  242,
  506,
  563,
  576,
  637,
  952,
  1051,
  1617,
  2231,
  2538,
  2656,
  4960,
  16429,
  16453,
  16475,
  16494,
  16540,
  16599,
  16620,
  17058,
  17081,
  17299,
  17365,
  19826,
  19978,
  22133,
  25079,
  26783,
  26856,
  26893,
  27339,
  27516,
  27776,
  29810,
  32008,
  32796],
 [7],
 [8,
  82,
  279,
  408,
  455,
  522,
  544,
  550,
  581,
  1025,
  6079,
  15970,
  15971,
  15981,
  16118,
  16126,
  16302,
  16494,
  16499,
  16751,
  17178,
  17361,
  23164,
  23284,
  26928,
  26960,
  27012,
  27262,
  27966,
  28500,
  29301,
  30405,
  30535,
  31400,
  31537,
  31703,
  32838,
  33185],
 [9],
 [10,
  21,
  30,
  41,
  47,
  59,
  261,
  379,
  381,
  410,
  411,
  416,
  436,
  437,
  454,
  492,
  498,
  510,
  538,
  558,
  615,
 

In [127]:
## 0.9이상 비슷한 특허가 없는 특허 인덱스 뽑기 = 자기 자신의 위치
similar_pat_list_self = [i for i,c in enumerate(similar_pat_list) if len(c)==1]

for i in similar_pat_list_self:
    similar_pat_list[i] = ['NoSimilarPatent']

In [134]:
similar_pat_list[:7]

[['NoSimilarPatent'],
 ['NoSimilarPatent'],
 [2, 98, 1617, 2656],
 [3,
  98,
  106,
  468,
  952,
  1051,
  2326,
  2656,
  16083,
  16111,
  16429,
  17299,
  19978,
  21279,
  22176,
  32796],
 ['NoSimilarPatent'],
 ['NoSimilarPatent'],
 [2,
  6,
  98,
  100,
  106,
  169,
  197,
  242,
  506,
  563,
  576,
  637,
  952,
  1051,
  1617,
  2231,
  2538,
  2656,
  4960,
  16429,
  16453,
  16475,
  16494,
  16540,
  16599,
  16620,
  17058,
  17081,
  17299,
  17365,
  19826,
  19978,
  22133,
  25079,
  26783,
  26856,
  26893,
  27339,
  27516,
  27776,
  29810,
  32008,
  32796]]

In [147]:
similar_pat_list[2]

[2, 98, 1617, 2656]

In [136]:
len(similar_pat_list)

33217

In [139]:
## 타겟특허의 cpc를 포함한 특허의 개수 (자기자신만 있는건 제거함) --> 분모
similar_pat_length = [(len(i)-1) for i in similar_pat_list]
similar_pat_length[:7]

[0, 0, 3, 15, 0, 0, 42]

In [156]:
pat_num_claims = list(final_patent_df['num_claims'])
pat_num_claims[:7]

[20, 18, 31, 9, 'NoClaimText', 20, 9]

In [161]:
## NoClaimText를 0으로 바꿈
pat_num_claims_=[]
for i in pat_num_claims:
    temp = str(i).replace("NoClaimText", '0')
    pat_num_claims_.append(temp)
    
pat_num_claims_ = [int(i) for i in pat_num_claims_]
pat_num_claims_[:7]

[20, 18, 31, 9, 0, 20, 9]

In [190]:
for a in similar_pat_list:
    similar_pat_num_claims=[]
    for b in a:
        if b == 'NoSimilarPatent':
            pass
        else:
            print(util_patent[b]['num_claims'])

SyntaxError: unexpected EOF while parsing (<ipython-input-190-309e6875836c>, line 7)

In [191]:
total_similar_pat_num_claims=[]
for a in similar_pat_list:
    similar_pat_num_claims=[]
    for b in a:
        if b == 'NoSimilarPatent':
            similar_pat_num_claims.append(0)
        else:
            similar_pat_num_claims.append(util_patent[b]['num_claims'])
    
    total_similar_pat_num_claims.append(similar_pat_num_claims)    

In [177]:
total_similar_pat_num_claims

[[0],
 [0],
 [31, 'NoClaimText', 'NoClaimText', 'NoClaimText'],
 [9,
  'NoClaimText',
  30,
  'NoClaimText',
  25,
  'NoClaimText',
  'NoClaimText',
  'NoClaimText',
  16,
  33,
  'NoClaimText',
  20,
  19,
  'NoClaimText',
  20,
  'NoClaimText'],
 [0],
 [0],
 [31,
  9,
  'NoClaimText',
  'NoClaimText',
  30,
  19,
  'NoClaimText',
  'NoClaimText',
  'NoClaimText',
  'NoClaimText',
  10,
  'NoClaimText',
  25,
  'NoClaimText',
  'NoClaimText',
  'NoClaimText',
  39,
  'NoClaimText',
  16,
  'NoClaimText',
  10,
  20,
  'NoClaimText',
  'NoClaimText',
  'NoClaimText',
  8,
  'NoClaimText',
  'NoClaimText',
  20,
  'NoClaimText',
  22,
  19,
  18,
  19,
  'NoClaimText',
  'NoClaimText',
  26,
  14,
  14,
  'NoClaimText',
  17,
  20,
  'NoClaimText']]

In [192]:
## NoClaimText를 0으로 바꿈
total_similar_pat_num_claims_=[]
for i in tqdm(total_similar_pat_num_claims):
    similar_pat_num_claims_ = []
    for j in i:
        temp = str(j).replace("NoClaimText", '0')
        similar_pat_num_claims_.append(temp)
    total_similar_pat_num_claims_.append(similar_pat_num_claims_)
    
total_similar_pat_num_claims__ = []
for i in tqdm(total_similar_pat_num_claims_):
    int_list = list(map(int, i))
    total_similar_pat_num_claims__.append(int_list)

100%|█████████████████████████████████████████████████████████████████████████| 33217/33217 [00:00<00:00, 35350.95it/s]


In [193]:
sum_similar_pat_num_claims = []
for i in tqdm(total_similar_pat_num_claims__):
    sum_similar_pat_num_claims.append(sum(i))

100%|████████████████████████████████████████████████████████████████████████| 33217/33217 [00:00<00:00, 443184.68it/s]


In [189]:
sum_similar_pat_num_claims

[0, 0, 31, 172, 0, 0, 406]

In [195]:
mean_num_claim = []
for i, n in enumerate(tqdm(similar_pat_length)):
    try:
        a = sum_similar_pat_num_claims[i]/n
        mean_num_claim.append(a)
    except ZeroDivisionError:
        mean_num_claim.append(0)

100%|███████████████████████████████████████████████████████████████████████| 33217/33217 [00:00<00:00, 2187711.14it/s]


In [197]:
len(mean_num_claim)

33217

In [198]:
final_patent_df

,patent_number,patent_title,patent_date,patent_type,patent_abstract,assignees,cited_patents,citedby_patents,examiners,cpc_set,...,backward_cpcs,originality,radicalness,broad_impact,grant_date,IKA,num_npl,examiner_impact,IKC,num_claims
0,9811636,Connected health care system,2017-11-07,utility,A connected health care system is provided wit...,[{'assignee_id': 'c6b8e6ee-b632-42c8-8f53-4d93...,"[{'cited_patent_number': None, 'cited_patent_d...","[{'citedby_patent_number': '10076306', 'citedb...","[{'examiner_id': '7djp2yknctv5h5pivqpgju4k2', ...","[A46B15, A46B15, A46B15, A46B15, A46B5, A46B22...",...,[],0.807889,No BC,0.351351,0.0,18,NaN,NaN,286,20
1,10064582,Noninvasive determination of cardiac health an...,2018-09-04,utility,This document describes assessment of human ph...,[{'assignee_id': '0bbbfb76-3528-467c-97dc-6d54...,"[{'cited_patent_number': '5410471', 'cited_pat...","[{'citedby_patent_number': '10376195', 'citedb...","[{'examiner_id': '3tpj24nwl1n2flyx4i5kqgeny', ...","[A47K13, A61B5, A61B5, A61B5, A61B5, A61B5, A6...",...,"['5410471', '5798798', '6254544', '6386757', '...",0.798334,0.353086,0.634146,NaN,2,1.0,NaN,228,18
2,10827928,Real-time monitoring systems and methods in a ...,2020-11-10,utility,An apparatus for real time monitoring of a pat...,[{'assignee_id': '9251c0e2-7d30-4044-bf81-97a0...,"[{'cited_patent_number': '6374371', 'cited_pat...","[{'citedby_patent_number': None, 'citedby_pate...","[{'examiner_id': 'm7qdml3up2oslhidj79c0baom', ...","[A61B5, A61B5, A61B5, A61B5, A61B5, A61B5, A61...",...,"['6374371', '6882883', '7229409', '8002701', '...",1.000000,0.295082,No FC,NaN,5,NaN,NaN,247,31
3,10052023,"Floor mat system and associated, computer medi...",2018-08-21,utility,"Provided are embodiments of systems, computer ...",[{'assignee_id': '827c2dfa-4040-4d6d-bc60-a8d4...,"[{'cited_patent_number': '4951963', 'cited_pat...","[{'citedby_patent_number': '10769574', 'citedb...","[{'examiner_id': '1ppzmlnxzqlegochzvy0mowbe', ...","[A61B5, A61B5, A61B5, A61B5, A61B5, A61B5, A61...",...,"['4951963', '4998534', '5000188', '5111539', '...",0.000000,0.566337,1,NaN,15,3.0,NaN,163,9
4,10206625,"Chair pad system and associated, computer medi...",2019-02-19,utility,"Provided are embodiments of systems, computer ...",[{'assignee_id': '827c2dfa-4040-4d6d-bc60-a8d4...,"[{'cited_patent_number': '4951963', 'cited_pat...","[{'citedby_patent_number': '10769574', 'citedb...","[{'examiner_id': 'asxak897kt9a7cqtxv2syvv4m', ...","[A61B5, A61B5, A61B5, A61B5, A61B5, A61B5, A61...",...,"['4951963', '4998534', '5000188', '5111539', '...",0.000000,0.571287,1,NaN,16,6.0,NaN,0,NoClaimText
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33212,10586614,System and method for timely multi-channel not...,2020-03-10,utility,A computer-assisted method to timely provide n...,[{'assignee_id': '3cb52395-6f90-4137-a8a4-0766...,"[{'cited_patent_number': None, 'cited_patent_d...","[{'citedby_patent_number': None, 'citedby_pate...","[{'examiner_id': '7fvqd8x22jc9k81zwryk9m3wr', ...","[G06N3, G06N20, G06N3, G06N5, G16H10, G16H50]",...,[],1.000000,No BC,No FC,0.0,7,NaN,NaN,428,18
33213,10695902,Pharmacy order processing system,2020-06-30,utility,A pharmacy order processing system includes a ...,[{'assignee_id': 'ba0fcc10-50b2-446f-8811-bb06...,"[{'cited_patent_number': '4944647', 'cited_pat...","[{'citedby_patent_number': None, 'citedby_pate...","[{'examiner_id': 'xrvecakaq07dvhhoe9d7yy7z0', ...","[B25J9, B25J11, B25J9, B25J9, B25J9, B25J18, B...",...,"['4944647', '5720154', '5771657', '6026561', '...",1.000000,0.910180,No FC,NaN,20,NaN,NaN,404,30
33214,10894027,Sickle cell disease treatment utilizing omega-...,2021-01-19,utility,Described herein are compositions including at...,[{'assignee_id': '63f3230c-69aa-4acc-9acb-a1bd...,"[{'cited_patent_number': '4513008', 'cited_pat...","[{'citedby_patent_number': None, 'citedby_pate...","[{'examiner_id': '3bxgj64hbifpvt5sxeowzom1x', ...","[A61K31, A61K9, A61K9, A61K31, A61K3

In [200]:
final_patent_df['under_examination_claim_index'] = mean_num_claim
final_patent_df

,patent_number,patent_title,patent_date,patent_type,patent_abstract,assignees,cited_patents,citedby_patents,examiners,cpc_set,...,originality,radicalness,broad_impact,grant_date,IKA,num_npl,examiner_impact,IKC,num_claims,under_examination_claim_index
0,9811636,Connected health care system,2017-11-07,utility,A connected health care system is provided wit...,[{'assignee_id': 'c6b8e6ee-b632-42c8-8f53-4d93...,"[{'cited_patent_number': None, 'cited_patent_d...","[{'citedby_patent_number': '10076306', 'citedb...","[{'examiner_id': '7djp2yknctv5h5pivqpgju4k2', ...","[A46B15, A46B15, A46B15, A46B15, A46B5, A46B22...",...,0.807889,No BC,0.351351,0.0,18,NaN,NaN,286,20,0.000000
1,10064582,Noninvasive determination of cardiac health an...,2018-09-04,utility,This document describes assessment of human ph...,[{'assignee_id': '0bbbfb76-3528-467c-97dc-6d54...,"[{'cited_patent_number': '5410471', 'cited_pat...","[{'citedby_patent_number': '10376195', 'citedb...","[{'examiner_id': '3tpj24nwl1n2flyx4i5kqgeny', ...","[A47K13, A61B5, A61B5, A61B5, A61B5, A61B5, A6...",...,0.798334,0.353086,0.634146,NaN,2,1.0,NaN,228,18,0.000000
2,10827928,Real-time monitoring systems and methods in a ...,2020-11-10,utility,An apparatus for real time monitoring of a pat...,[{'assignee_id': '9251c0e2-7d30-4044-bf81-97a0...,"[{'cited_patent_number': '6374371', 'cited_pat...","[{'citedby_patent_number': None, 'citedby_pate...","[{'examiner_id': 'm7qdml3up2oslhidj79c0baom', ...","[A61B5, A61B5, A61B5, A61B5, A61B5, A61B5, A61...",...,1.000000,0.295082,No FC,NaN,5,NaN,NaN,247,31,10.333333
3,10052023,"Floor mat system and associated, computer medi...",2018-08-21,utility,"Provided are embodiments of systems, computer ...",[{'assignee_id': '827c2dfa-4040-4d6d-bc60-a8d4...,"[{'cited_patent_number': '4951963', 'cited_pat...","[{'citedby_patent_number': '10769574', 'citedb...","[{'examiner_id': '1ppzmlnxzqlegochzvy0mowbe', ...","[A61B5, A61B5, A61B5, A61B5, A61B5, A61B5, A61...",...,0.000000,0.566337,1,NaN,15,3.0,NaN,163,9,11.466667
4,10206625,"Chair pad system and associated, computer medi...",2019-02-19,utility,"Provided are embodiments of systems, computer ...",[{'assignee_id': '827c2dfa-4040-4d6d-bc60-a8d4...,"[{'cited_patent_number': '4951963', 'cited_pat...","[{'citedby_patent_number': '10769574', 'citedb...","[{'examiner_id': 'asxak897kt9a7cqtxv2syvv4m', ...","[A61B5, A61B5, A61B5, A61B5, A61B5, A61B5, A61...",...,0.000000,0.571287,1,NaN,16,6.0,NaN,0,NoClaimText,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33212,10586614,System and method for timely multi-channel not...,2020-03-10,utility,A computer-assisted method to timely provide n...,[{'assignee_id': '3cb52395-6f90-4137-a8a4-0766...,"[{'cited_patent_number': None, 'cited_patent_d...","[{'citedby_patent_number': None, 'citedby_pate...","[{'examiner_id': '7fvqd8x22jc9k81zwryk9m3wr', ...","[G06N3, G06N20, G06N3, G06N5, G16H10, G16H50]",...,1.000000,No BC,No FC,0.0,7,NaN,NaN,428,18,9.500000
33213,10695902,Pharmacy order processing system,2020-06-30,utility,A pharmacy order processing system includes a ...,[{'assignee_id': 'ba0fcc10-50b2-446f-8811-bb06...,"[{'cited_patent_number': '4944647', 'cited_pat...","[{'citedby_patent_number': None, 'citedby_pate...","[{'examiner_id': 'xrvecakaq07dvhhoe9d7yy7z0', ...","[B25J9, B25J11, B25J9, B25J9, B25J9, B25J18, B...",...,1.000000,0.910180,No FC,NaN,20,NaN,NaN,404,30,0.000000
33214,10894027,Sickle cell disease treatment utilizing omega-...,2021-01-19,utility,Described herein are compositions including at...,[{'assignee_id': '63f3230c-69aa-4acc-9acb-a1bd...,"[{'cited_patent_number': '4513008', 'cited_pat...","[{'citedby_patent_number': None, 'citedby_pate...","[{'examiner_id': '3bxgj64hbifpvt5sxeowzom1x', ...","[A61K31, A61K9, A61K9, A61K31, A61K31, A61K31,...",...,1.000000,0.790960,No FC,NaN,10,NaN,NaN,0,NoClaimText,9.171196
33215,10643746,Image viewer,2020-05-05,utility,An image viewer and method for using the same ...,[{'assi

In [201]:
final_patent_df.to_excel('./data/final_patent.xlsx', encoding='utf8', index=False)

In [227]:
final_patent_df = pd.read_excel('./data/final_patent.xlsx')

In [228]:
util_patent = final_patent_df.to_dict('records')

In [230]:
assignee_df = final_patent_df[['patent_number', 'assignees', 'cpc_set']]
assignee_df

,patent_number,assignees,cpc_set
0,9811636,[{'assignee_id': 'c6b8e6ee-b632-42c8-8f53-4d93...,"['A46B15', 'A46B15', 'A46B15', 'A46B15', 'A46B..."
1,10064582,[{'assignee_id': '0bbbfb76-3528-467c-97dc-6d54...,"['A47K13', 'A61B5', 'A61B5', 'A61B5', 'A61B5',..."
2,10827928,[{'assignee_id': '9251c0e2-7d30-4044-bf81-97a0...,"['A61B5', 'A61B5', 'A61B5', 'A61B5', 'A61B5', ..."
3,10052023,[{'assignee_id': '827c2dfa-4040-4d6d-bc60-a8d4...,"['A61B5', 'A61B5', 'A61B5', 'A61B5', 'A61B5', ..."
4,10206625,[{'assignee_id': '827c2dfa-4040-4d6d-bc60-a8d4...,"['A61B5', 'A61B5', 'A61B5', 'A61B5', 'A61B5', ..."
...,...,...,...
33212,10586614,[{'assignee_id': '3cb52395-6f90-4137-a8a4-0766...,"['G06N3', 'G06N20', 'G06N3', 'G06N5', 'G16H10'..."
33213,10695902,[{'assignee_id': 'ba0fcc10-50b2-446f-8811-bb06...,"['B25J9', 'B25J11', 'B25J9', 'B25J9', 'B25J9',..."
33214,10894027,[{'assignee_id': '63f3230c-69aa-4acc-9acb-a1bd...,"['A61K31', 'A61K9', 'A61K9', 'A61K31', 'A61K31..."
33215,10643746,[{'assignee_id': '720a62eb-e056-40c7-88c9-176e...,"['G06T1', 'G06T1', 'G06T2200', 'G16H30', 'G16H..."


In [ ]:
### heterogeneity

assignee_util_patent = assignee_df.to_dict('records')
assignee_util_patent[0]

# 리스트 모양의 문자열을 실제 리스트로 바꿈
from ast import literal_eval
for p in assignee_util_patent:
    p['assignees'] = literal_eval(p['assignees'])
    p['cpc_set'] = literal_eval(p['cpc_set'])

# assignee_id만 추출
assignee_id = []
for a in assignee_util_patent:
    assignee_id.append(a['assignees'][0]['assignee_id'])

    
# (출원인의 전체 특허수 - 수집한 특허에서 출원인이 가진 특허수)/출원인의 전체 특허수
heterogeneity = []
for a in tqdm(assignee_util_patent):
    num_pats = assignee_id.count(a['assignees'][0]['assignee_id'])
    try:
        total_pats_of_assignee = int(a['assignees'][0]['assignee_total_num_patents'])
    except TypeError:
        total_pats_of_assignee = 0
    
    try:
        h = ((total_pats_of_assignee)-num_pats)/total_pats_of_assignee
    except ZeroDivisionError:
        h = 0
    heterogeneity.append(h)

In [231]:
assignee_util_patent = assignee_df.to_dict('records')
assignee_util_patent[0]

{'patent_number': 9811636,
 'assignees': "[{'assignee_id': 'c6b8e6ee-b632-42c8-8f53-4d93c81a6b50', 'assignee_total_num_patents': '16', 'assignee_key_id': '48452'}]",
 'cpc_set': "['A46B15', 'A46B15', 'A46B15', 'A46B15', 'A46B5', 'A46B2200', 'A61C19', 'A61B5', 'A61B5', 'A61B5', 'A61B5', 'A61B5', 'A61C17', 'A61C17', 'A61B5', 'A61C17', 'G06Q10', 'G06Q40', 'G16H40', 'G16H50']"}

In [232]:
from ast import literal_eval
for p in assignee_util_patent:
    p['assignees'] = literal_eval(p['assignees'])
    p['cpc_set'] = literal_eval(p['cpc_set'])

In [233]:
assignee_util_patent[0]

{'patent_number': 9811636,
 'assignees': [{'assignee_id': 'c6b8e6ee-b632-42c8-8f53-4d93c81a6b50',
   'assignee_total_num_patents': '16',
   'assignee_key_id': '48452'}],
 'cpc_set': ['A46B15',
  'A46B15',
  'A46B15',
  'A46B15',
  'A46B5',
  'A46B2200',
  'A61C19',
  'A61B5',
  'A61B5',
  'A61B5',
  'A61B5',
  'A61B5',
  'A61C17',
  'A61C17',
  'A61B5',
  'A61C17',
  'G06Q10',
  'G06Q40',
  'G16H40',
  'G16H50']}

In [285]:
assignee_util_patent[0]['assignees'][0]['assignee_id']

'c6b8e6ee-b632-42c8-8f53-4d93c81a6b50'

In [289]:
assignee_id = []
for a in assignee_util_patent:
    assignee_id.append(a['assignees'][0]['assignee_id'])

In [291]:
assignee_util_patent[8]

{'patent_number': 10699356,
 'assignees': [{'assignee_id': None,
   'assignee_total_num_patents': None,
   'assignee_key_id': None}],
 'cpc_set': ['G06Q10', 'G16H10', 'G16H20', 'G16H40', 'G16H50', 'G16H50']}

In [290]:
assignee_id

['c6b8e6ee-b632-42c8-8f53-4d93c81a6b50',
 '0bbbfb76-3528-467c-97dc-6d54efa5dc66',
 '9251c0e2-7d30-4044-bf81-97a06f985f5b',
 '827c2dfa-4040-4d6d-bc60-a8d4d9aafe2d',
 '827c2dfa-4040-4d6d-bc60-a8d4d9aafe2d',
 '827c2dfa-4040-4d6d-bc60-a8d4d9aafe2d',
 '827c2dfa-4040-4d6d-bc60-a8d4d9aafe2d',
 '9a47a7fa-78a6-4f8a-8f17-af732f6a364a',
 None,
 '5c398d6c-13d7-4028-afb1-835ef465ac61',
 '15a09840-f4d9-45b5-95d1-9815d53d66e0',
 None,
 '332e1e37-1a83-485f-8f6e-a3737c7a3ad1',
 'bf5369d3-8253-4576-a383-2bddebb8a381',
 '906a245b-a24e-409e-9d8f-298daab7c9b3',
 '40a9941d-f601-4153-8c63-fc5e722266d6',
 None,
 '9226f064-dd25-489a-971f-0ba359a01374',
 'd18bed0f-dd50-41ca-a968-87844d8c73ec',
 'cee06533-4be7-41b9-88ab-7f97f20c15be',
 '6bf27dfc-9a42-413d-9f34-245abd828a84',
 '1ace15f5-5ade-4080-a1c5-65f83180b5e8',
 None,
 '3b2e6a9b-d90e-4095-8457-1965a12499e1',
 '2f71c7d6-9b4a-44ae-84b7-6c88d872871a',
 '4961df1c-11c7-4c5b-adba-d90dcf617b89',
 'f3b466e0-56bf-4ed1-ac9c-f9908bbbefa2',
 '70e783ec-2c55-4df7-851a-737

In [328]:
assignee_id.count('0bbbfb76-3528-467c-97dc-6d54efa5dc66')

57

In [332]:
assignee_id[:11]

['c6b8e6ee-b632-42c8-8f53-4d93c81a6b50',
 '0bbbfb76-3528-467c-97dc-6d54efa5dc66',
 '9251c0e2-7d30-4044-bf81-97a06f985f5b',
 '827c2dfa-4040-4d6d-bc60-a8d4d9aafe2d',
 '827c2dfa-4040-4d6d-bc60-a8d4d9aafe2d',
 '827c2dfa-4040-4d6d-bc60-a8d4d9aafe2d',
 '827c2dfa-4040-4d6d-bc60-a8d4d9aafe2d',
 '9a47a7fa-78a6-4f8a-8f17-af732f6a364a',
 None,
 '5c398d6c-13d7-4028-afb1-835ef465ac61',
 '15a09840-f4d9-45b5-95d1-9815d53d66e0']

In [331]:
assignee_util_patent[0]['assignees'][0]['assignee_total_num_patents']

'16'

In [333]:
assignee_util_patent[0]['assignees'][0]['assignee_id']

'c6b8e6ee-b632-42c8-8f53-4d93c81a6b50'

In [335]:
assignee_util_patent[0]

{'patent_number': 9811636,
 'assignees': [{'assignee_id': 'c6b8e6ee-b632-42c8-8f53-4d93c81a6b50',
   'assignee_total_num_patents': '16',
   'assignee_key_id': '48452'}],
 'cpc_set': ['A46B15',
  'A46B15',
  'A46B15',
  'A46B15',
  'A46B5',
  'A46B2200',
  'A61C19',
  'A61B5',
  'A61B5',
  'A61B5',
  'A61B5',
  'A61B5',
  'A61C17',
  'A61C17',
  'A61B5',
  'A61C17',
  'G06Q10',
  'G06Q40',
  'G16H40',
  'G16H50']}

In [340]:
heterogeneity = []
for a in tqdm(assignee_util_patent):
    num_pats = assignee_id.count(a['assignees'][0]['assignee_id'])
    try:
        total_pats_of_assignee = int(a['assignees'][0]['assignee_total_num_patents'])
    except TypeError:
        total_pats_of_assignee = 0
    
    try:
        h = ((total_pats_of_assignee)-num_pats)/total_pats_of_assignee
    except ZeroDivisionError:
        h = 0
    heterogeneity.append(h)

100%|██████████████████████████████████████████████████████████████████████████| 33217/33217 [00:16<00:00, 2059.98it/s]


In [351]:
final_patent_df['heterogeneity'] = heterogeneity
final_patent_df

,patent_number,patent_title,patent_date,patent_type,patent_abstract,assignees,cited_patents,citedby_patents,examiners,cpc_set,cited_patent_set,num_cited,citedby_patent_set,num_citedby,median_grant_date,examiner_type,forward_cpcs,AAFC,backward_cpcs,originality,radicalness,broad_impact,grant_date,IKA,num_npl,examiner_impact,IKC,num_claims,under_examination_claim_index,heterogeneity
0,9811636,Connected health care system,2017-11-07,utility,A connected health care system is provided wit...,[{'assignee_id': 'c6b8e6ee-b632-42c8-8f53-4d93...,"[{'cited_patent_number': None, 'cited_patent_d...","[{'citedby_patent_number': '10076306', 'citedb...","[{'examiner_id': '7djp2yknctv5h5pivqpgju4k2', ...","['A46B15', 'A46B15', 'A46B15', 'A46B15', 'A46B...",None,1,",10076306,10201701,10869741",3,0.0,0,"['10076306', '10201701', '10869741']",0.750000,[],0.807889,No BC,0.351351,0.0,18,NaN,NaN,286,20,0.000000,0.687500
1,10064582,Noninvasive determination of cardiac health an...,2018-09-04,utility,This document describes assessment of human ph...,[{'assignee_id': '0bbbfb76-3528-467c-97dc-6d54...,"[{'cited_patent_number': '5410471', 'cited_pat...","[{'citedby_patent_number': '10376195', 'citedb...","[{'examiner_id': '3tpj24nwl1n2flyx4i5kqgeny', ...","['A47K13', 'A61B5', 'A61B5', 'A61B5', 'A61B5',...",",5410471,5798798,6254544,6386757,6513970,65242...",27,",10376195,10936081,10936085,10948996",4,2011.0,0,"['10376195', '10936081', '10936085', '10948996']",1.333333,"['5410471', '5798798', '6254544', '6386757', '...",0.798334,0.353086,0.634146,NaN,2,1.0,NaN,228,18,0.000000,0.997582
2,10827928,Real-time monitoring systems and methods in a ...,2020-11-10,utility,An apparatus for real time monitoring of a pat...,[{'assignee_id': '9251c0e2-7d30-4044-bf81-97a0...,"[{'cited_patent_number': '6374371', 'cited_pat...","[{'citedby_patent_number': None, 'citedby_pate...","[{'examiner_id': 'm7qdml3up2oslhidj79c0baom', ...","['A61B5', 'A61B5', 'A61B5', 'A61B5', 'A61B5', ...",",6374371,6882883,7229409,8002701,8310374,84877...",9,None,1,2012.0,1,[],0,"['6374371', '6882883', '7229409', '8002701', '...",1.000000,0.295082,No FC,NaN,5,NaN,NaN,247,31,10.333333,0.000000
3,10052023,"Floor mat system and associated, computer medi...",2018-08-21,utility,"Provided are embodiments of systems, computer ...",[{'assignee_id': '827c2dfa-4040-4d6d-bc60-a8d4...,"[{'cited_patent_number': '4951963', 'cited_pat...","[{'citedby_patent_number': '10769574', 'citedb...","[{'examiner_id': '1ppzmlnxzqlegochzvy0mowbe', ...","['A61B5', 'A61B5', 'A61B5', 'A61B5', 'A61B5', ...",",4951963,4998534,5000188,5111539,5253656,53041...",136,",10769574",1,2005.0,1,['10769574'],0.333333,"['4951963', '4998534', '5000188', '5111539', '...",0.000000,0.566337,1,NaN,15,3.0,NaN,163,9,11.466667,0.995723
4,10206625,"Chair pad system and associated, computer medi...",2019-02-19,utility,"Provided are embodiments of systems, computer ...",[{'assignee_id': '827c2dfa-4040-4d6d-bc60-a8d4...,"[{'cited_patent_number': '4951963', 'cited_pat...","[{'citedby_patent_number': '10769574', 'citedb...","[{'examiner_id': 'asxak897kt9a7cqtxv2syvv4m', ...","['A61B5', 'A61B5', 'A61B5', 'A61B5', 'A61B5', ...",",4951963,4998534,5000188,5111539,5253656,53041...",136,",10769574",1,2005.0,1,['10769574'],0.500000,"['4951963', '4998534', '5000188', '5111539', '...",0.000000,0.571287,1,NaN,16,6.0,NaN,0,NoClaimText,0.000000,0.995723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33212,10586614,System and method for timely multi-channel not...,2020-03-10,utility,A computer-assisted method to timely provide n...,[{'assignee_id': '3cb52395-6f90-4137-a8a4-0766...,"[{'cited_patent_number': None, 'cited_patent_d...","[{'citedby_patent_number': None, 'citedby_pate...","[{'examiner_id': '7fvqd8x22jc9k81zwryk9m3wr', ...","['G06N3', 'G06N20', 'G06N3', 'G06N5', 'G16H10'...",None,1,None,1,0.0,1,[],0,[],1.000000,No BC,No FC,0.0,7,NaN,NaN,428,18,9.500000,0.583333
33213,10695902,Pharm

In [352]:
final_patent_df.to_excel('./data/final_patent.xlsx', encoding='utf8', index=False)

In [337]:
assignee_util_patent[8]

{'patent_number': 10699356,
 'assignees': [{'assignee_id': None,
   'assignee_total_num_patents': None,
   'assignee_key_id': None}],
 'cpc_set': ['G06Q10', 'G16H10', 'G16H20', 'G16H40', 'G16H50', 'G16H50']}

In [338]:
assignee_id.count(assignee_util_patent[8]['assignees'][0]['assignee_id'])

2079

In [361]:
forward_patent_df = pd.read_csv('./data/forward_citation_cpcs.csv', encoding='utf8')

C:\Users\Woojin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6,7,8,9,10,11,12,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [362]:
forward_util_patent = forward_patent_df.to_dict('records')

In [359]:
util_patent[0]['citedby_patents']

[{'citedby_patent_number': '10076306', 'citedby_patent_date': '2018-09-18'},
 {'citedby_patent_number': '10201701', 'citedby_patent_date': '2019-02-12'},
 {'citedby_patent_number': '10869741', 'citedby_patent_date': '2020-12-22'}]

In [358]:
util_patent[0]['citedby_patents'][0]

{'citedby_patent_number': '10076306', 'citedby_patent_date': '2018-09-18'}

In [380]:
final_patent_df['patent_abstract'][0]

'A connected health care system is provided with various devices for tracking and monitoring health statistics and behaviors including oral health, fitness, heart health, bone health, salivary diagnostics, diabetes, and further options that will be explored in the subsequent sections. The connected health care system is comprised of various devices, a data transfer medium (i.e. “smartphone”), and the Cloud, which allows for data transfer between multiple platforms and devices. Some devices comprised in the connected health care system include a toothbrush, a connected surface with sensors, a salivary diagnostic system, a breath analysis system, and further devices that will be explored in the subsequent sections. The connected health care system further provides a user identification system utilizing capacitive coupling of the human body between devices.'